In [13]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import re
import gzip

In [14]:
import pickle

In [15]:
PATH_WIKI_XML = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki_20180920/xml+art+red/'
PATH_WIKI_OUT = '/media/andrii/earth/Katia/CS_MasterThesis/data/ukwiki/ukwiki_20180920/'

In [16]:
WIKI_FILENAMES = []
for file in os.listdir(PATH_WIKI_XML):
    if file.endswith(".xml"):
        WIKI_FILENAMES.append(file)

In [17]:
WIKI_FILENAMES

['ukwiki-20180920-pages-meta-current.xml', 'test.xml']

In [18]:
WIKI_FILENAMES = [WIKI_FILENAMES[0]]

In [19]:
WIKI_FILENAMES

['ukwiki-20180920-pages-meta-current.xml']

In [20]:
ENCODING = "utf-8"


# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [21]:
totalCount = 0
articleCount = 0
redirectCount = 0
total_article_text_len = 0

In [ ]:
(?P<context1>(\S+\s+){10})\[\[(?P<article>.*?)(?:\{\{.*\}\})?(?:\|(?P<text>.*?))?\]\](?P<context2>(\s+\S+){10})

In [22]:
#\[\[(?P<article>(?!\S*?\:).*?)(?:\{\{.*\}\})?(?:\|(?P<text>(?!\S*?\:).*?))?\]\]
start_time = time.time()
regex_links = re.compile(r"(?P<precontext>(?:[a-zA-Z'-]+[^a-zA-Z'-]+){,10})(\[\[(?P<article>.*?)(?:\{\{.*\}\})?(?:\|(?P<text>.*?))?\]\])(?P<postcontext>([^a-zA-Z'-]+[a-zA-Z'-]+){,10})")
for WikiXML in WIKI_FILENAMES:
    pathWikiXML = os.path.join(PATH_WIKI_XML, WikiXML)
    pathArticles = os.path.join(PATH_WIKI_OUT, WikiXML+"_art.csv")
    pathArticlesRedirect = os.path.join(PATH_WIKI_OUT, WikiXML+"_red.csv")
    with codecs.open(pathArticles, "w", ENCODING) as articlesFH, \
    codecs.open(pathArticlesRedirect, "w", ENCODING) as redirectFH:
        articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
        articlesWriter.writerow(['id', 'title', 'text_len', 'link_pos', 'link_val', 'link_txt', 'link_precontext', 'link_postcontext']) 
        redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)
        redirectWriter.writerow(['id', 'title', 'redirect'])
        for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
            tname = strip_tag_name(elem.tag)

            if event == 'start':
                if tname == 'page':
                    title = ''
                    id = -1
                    redirect = ''
                    inrevision = False
                    ns = 0
                    article_text_len = 0
                    links = []
                elif tname == 'revision':
                    # Do not pick up on revision id's
                    inrevision = True
            else:
                if tname == 'title':
                    title = elem.text
                elif tname == 'id' and not inrevision:
                    id = int(elem.text)
                elif tname == 'redirect':
                    redirect = elem.attrib['title']
                elif tname == 'ns':
                    ns = int(elem.text)
                elif tname == 'page' and ns == 0:
                    totalCount += 1
                        
                    if len(redirect) == 0:
                        articleCount += 1
                        total_article_text_len += article_text_len
                        if len(links) == 0:
                            articlesWriter.writerow([id, title, article_text_len, 0, "", ""])
                        for link in links:
                            articlesWriter.writerow([id, title, article_text_len, link[0], link[1], link[2], link[3], link[4]])
                    else:
                        redirectCount += 1
                        redirectWriter.writerow([id, title, redirect])

                    if totalCount > 1 and (totalCount % 100000) == 0:
                        print("{:,}".format(totalCount))
                elif tname == 'text' and elem.text != None:
                    article_text_len = len(elem.text)
                    for match in regex_links.finditer(elem.text):    
                        link_pos = match.start()

                        link_title = match.group("article")
                        link_title = link_title.replace("&nbsp;", " ")
                        link_title = link_title.replace("&ndash;", "-")
                        link_title = link_title.replace("&mdash;", "—")
                        link_title = link_title.replace("%20", " ")
                        
                        link_txt = match.group("text")
                        
                        link_precontext = match.group("precontext")
                        link_postcontext = match.group("postcontext")
                        
                        links.append((link_pos,link_title,link_txt,link_precontext,link_postcontext))


                elem.clear()
    
    fn_in = pathArticles
    with open(fn_in, 'rb') as f_in, gzip.open(fn_in+'.gz', 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(fn_in)
    
    fn_in = pathArticlesRedirect
    with open(fn_in, 'rb') as f_in, gzip.open(fn_in+'.gz', 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(fn_in)
    
    elapsed_time = time.time() - start_time
    print("File processed: {}".format(WikiXML))
    print("Total pages: {:,}".format(totalCount))
    print("Article pages: {:,}".format(articleCount))
    print("Redirect pages: {:,}".format(redirectCount))
    print("Total article lenght: {:,}".format(total_article_text_len))
    print("Elapsed time: {}".format(hms_string(elapsed_time)))

elapsed_time = time.time() - start_time

print("Total pages: {:,}".format(totalCount))
print("Article pages: {:,}".format(articleCount))
print("Redirect pages: {:,}".format(redirectCount))
print("Total article lenght: {:,}".format(total_article_text_len))
print("Elapsed time: {}".format(hms_string(elapsed_time)))

100,000
200,000
300,000
400,000
500,000
600,000
700,000
800,000
900,000
1,000,000
1,100,000
1,200,000
File processed: ukwiki-20180920-pages-meta-current.xml
Total pages: 1,272,760
Article pages: 819,212
Redirect pages: 453,548
Total article lenght: 3,879,541,085
Elapsed time: 0:52:33.37
Total pages: 1,272,760
Article pages: 819,212
Redirect pages: 453,548
Total article lenght: 3,879,541,085
Elapsed time: 0:52:33.37


In [40]:
import io
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [42]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /home/andrii/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [51]:
file1 = open(PATH_WIKI_XML+'test.xml')

In [52]:
line = file1.read() # Use this to read file content as a stream: 
words = line.split() 
for r in words: 
    if not r in stop_words: 
        appendFile = open(PATH_WIKI_XML+'filteredtext.xml','a') 
        appendFile.write(" "+r) 
        appendFile.close()

In [54]:
my_file = open(PATH_WIKI_XML+'filteredtext.xml')
my_string = my_file.read()
print(my_string)
my_file.close()

 <mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en"> <siteinfo> <sitename>Wikipedia</sitename> <dbname>enwiki</dbname> <base>https://en.wikipedia.org/wiki/Main_Page</base> <generator>MediaWiki 1.32.0-wmf.20</generator> <case>first-letter</case> <namespaces> <namespace key="-2" case="first-letter">Media</namespace> <namespace key="-1" case="first-letter">Special</namespace> <namespace key="0" case="first-letter" /> <namespace key="1" case="first-letter">Talk</namespace> <namespace key="2" case="first-letter">User</namespace> <namespace key="3" case="first-letter">User talk</namespace> <namespace key="4" case="first-letter">Wikipedia</namespace> <namespace key="5" case="first-letter">Wikipedia talk</namespace> <namespace key="6" case="first-letter">File</namespace> <namespace key="7" case="

In [31]:
with open(PATH_WIKI_XML+WIKI_FILENAMES[0], 'r') as test_f:
    test_contents = test_f.read()

In [42]:
with open(PATH_WIKI_XML+'ukwiki-20180920-pages-meta-current.xml', 'r') as file:
    contents = file.read()

MemoryError: 

In [32]:
regex_links = re.compile(r"((?:[a-zA-Z'-]+[^a-zA-Z'-]+){,10})(\[\[(.+?)(\|.+?)?\]\])")

In [42]:
matches = regex_links.finditer(test_contents)

In [43]:
n = 0
for link in matches:
    print(link.group(3))
    print()
    n += 1
print(n)

Computer accessibility

political philosophy

Self-governance

Stateless society

Free association (communism and anarchism)

State (polity)

The Globe and Mail

Routledge Encyclopedia of Philosophy

Anti-statism

Far-left politics

anarchist economics

Libertarian socialism

Mutualism (economic theory)

participatory economics

Anarchist schools of thought

collectivism

Social anarchism

individualist anarchism

wikt:anarchism

anarchy

-ism

Online etymology dictionary

privative

privative alpha

archon

infinitive

Online etymology dictionary

French Revolution

Hébertists

William Godwin

Wilhelm Weitling

Pierre-Joseph Proudhon

libertarianism

Joseph Dejacque

Anarchist Seeds Beneath the Snow

Dwight Macdonald

Alexandre Skirda

libertarian anarchism

market economy

File:Levellers declaration and standard.gif

William Everard (Digger)

Taoism

Laozi

Zhuang Zhou

wikt:brigand

Murray Rothbard

Journal of Libertarian Studies

Diogenes of Sinope

Zeno of Citium

Stoicism

Jesus


Right-libertarianism

Demanding the Impossible: A History of Anarchism

Saul Newman

right libertarianism

left libertarianism

Anarchist Seeds Beneath the Snow: Left-Libertarian Thought and British Writers from William Morris to Colin Ward

Anarchy: A Journal of Desire Armed

Anarcho-transhumanism

anarcho-syndicalism

left-libertarianism

transhumanism

post-humanism

technogaianism

anarchist schools of thought

File:Gadewar.jpg

philosophy

Anarchism and capitalism

Oxford University Press

Anarchism and religion

Anarchism and Marxism

Issues in anarchism#Communism

collectivism

syndicalism

trade unionism

God

enlightened self-interest

veganarchism

civilisation

technology

Issues in anarchism#Participation in statist democracy

propaganda of the deed

nihilist movement

nonviolence

counter-economics

Crypto-anarchism

definitional concerns in anarchist theory

File:Emilearmand01.jpg

Émile Armand

Anarchism in France

free love

Josiah Warren

women's rights

Moses Harman



Bucharest

Timişoara

Russia–Tunisia relations

Tunis

Moscow

Serbia

Valletta

Malta

Canberra

Farid Abboud

List of diplomatic missions in Tunisia

List of diplomatic missions of Tunisia

Category:Foreign relations of Tunisia

WP:AES

History of Turkey

History of Turkey

Europe

Western Asia

Land borders

Azerbaijan

Georgia (country)

Iran

Syria

Mount Ararat

Mediterranean Sea

Kızıl River

Lake Van

Anatolia

the Balkans

Black Sea

Aegean Sea

Syria

Turkey

Western Asia

Anatolia

Thrace

Anatolian Plateau

East Thrace

Istanbul

Bosphorus

Sea of Marmara

Dardanelles

Turkish Straits

Mount Ararat

summit (topography)

Noah's Ark

Greece

Bulgaria

Georgia (country)

Armenia

Azerbaijan

Iran

Iraq

Syria

maritime boundary

Greece

Treaty of Lausanne (1923)

Turkish War of Independence

Thrace

Balkan wars

Bulgaria

Treaty of Lausanne (1923)

Soviet Union

Treaty of Kars

Azerbaijan

Iran

Treaty of Zuhab

Iraq

Ankara

Syria

Ottoman Empire

Treaty of Lausanne (1923)

L

Bulgaria

Romania

Yalova Province

Refugees of the Syrian Civil War

Sephardi Jews

pogrom

Chechens

Trotskyists

History of the Jews in Turkey

Muhajir (Turkey)

Crimean Tatars

Bosniaks

Albanians

revolutions of 1989

Eastern Europe

USSR

Istanbul

Russian language

Russians in Turkey

Western Europe

Germany

United Kingdom

Netherlands

Ireland

Italy

United States

NUTS of Turkey

İstanbul Region (statistical)

West Marmara Region (statistical)

Aegean Region (statistical)

East Marmara Region (statistical)

West Anatolia Region (statistical)

Mediterranean Region (statistical)

Central Anatolia Region (statistical)

West Black Sea Region (statistical)

East Black Sea Region (statistical)

Northeast Anatolia (statistical)

Central East Anatolia (statistical)

Southeast Anatolia (statistical)

İstanbul Region (statistical)

West Marmara Region (statistical)

Aegean Region (statistical)

East Marmara Region (statistical)

West Anatolia Region (statistical)

Mediterranean Region

FAO

Anadolu Agency

NTV-MSNBC

hectares

Southeastern Anatolia Project

Fishing

fishery

aquaculture

European Union–Turkey Customs Union

European Union–Turkey relations

Ayvalık

Gemlik

Beko

consumer electronics

Vestel

Europe

Beko

CNN.com

Profilo Telra

Business Wire

European Union

consumer electronics

EU-Turkey Customs Union

clothing

File:Temsa Diamond Bus in Mannheim 100 7800.jpg

Otokar

BMC (Turkey)

Marmara Region

TÜLOMSAŞ

TÜVASAŞ

train

High-speed rail

Electric multiple unit

File:SOM cruise missile mockup on MSPO 2014.jpeg

TÜBİTAK Defense Industries Research and Development Institute

Turkish Air Force

Makina ve Kimya Endustrisi Kurumu

Turkish Aerospace Industries

Aselsan

Roketsan

FNSS Savunma Sistemleri A.Ş.

Nurol Makina

Otokar

Havelsan

F-35 Joint Strike Fighter

Turkish Aerospace Industries

F-16 Fighting Falcon

Turkish Air Force

reconnaissance satellite

Göktürk-1

Turkish Armed Forces

Göktürk-2

National Intelligence Organization (Turkey)

Al

...

Turks and Caicos Islands general election, 2016

People's Democratic Movement (Turks and Caicos Islands)

Sharlene Cartwright-Robinson

Turks and Caicos Islands House of Assembly

Judicial Committee of the Privy Council

John Fieldsend

Frederick Smith (barrister)

Gordon Ward

infant mortality

life expectancy

CIA World Factbook

Demographics of Europe

Indo-Caribbean

South Caicos

Cockburn Harbour

West Caicos

New Marina

Providenciales

Pine Cay

South Bay Village

Parrot Cay

Parrot Cay Village

North Caicos

Bottle Creek

Middle Caicos

Conch Bar Caves

Ambergris Cay

Ambergris Cay

East Caicos

Grand Turk Island

Cockburn Town

Salt Cay, Turks Islands

Salt Cay, Turks Islands

Cotton Cay

Cockburn Town

Turks and Caicos Islands Creole

Ethnologue

Bahamian Creole

Cuba

Haitian Creole

Haiti

Dominican Republic

Christian

Baptists

Church of God (Cleveland, Tennessee)

Roman Catholics

Anglicans

Methodism

Seventh-day Adventist Church

Jehovah's Witnesses

Roman Catholic Mis

Vaitupu

Fetuvalu Secondary School

Church of Tuvalu

International General Certificate of Secondary Education

University of the South Pacific

Tuvalu Maritime Training Institute

Bureau of International Labor Affairs

U.S. Department of Labor

File:Interior of a maneapa in Funafuti, Tuvalu.jpg

#Hedley

Hernandia

Pemphis

Thespesia populnea

Rhizophora mucronata

Hibiscus tiliaceus

coconut

Ficus

Pandanus

sennit

Maneaba

File:Dancer, Tuvalu stage, 2011 Pasifika festival.jpg

cowrie

Crochet

fatele

fakanau

fakaseasea

fatele

cuisine of Tuvalu

coconut

coconut milk

pulaka

taro

breadfruit

#Hedley

coconut crab

black noddy

white tern

carbohydrate

protein

Vaitupu

aquaculture in Tuvalu

Flying fish

File:Canoe carving on Nanumea.jpg

Maneaba

Falekaupule

cricket

pandanus

lance

quarterstaff

#Hedley

association football

futsal

volleyball

handball

basketball

Tuvalu Athletics Association

Tuvalu Rugby Union

Tuvalu Weightlifting Federation

Tuau Lapua Lapua

Snat

Project:AutoWikiBrowser

One Ring

Special:Contributions/31.53.1.161

User talk:Mornarben

User:GimliDotNet

User talk:GimliDotNet

User:Hairy Dude

User talk:Hairy Dude

User:Hairy Dude

User talk:Hairy Dude

Special:Contributions/199.168.151.168

User talk:199.168.151.168

List of Middle-earth animals

User:General Ization

User talk:General Ization

User talk:131.159.76.164

User:Lucifer &amp;#38; Viktor

User talk:Lucifer &amp;#38; Viktor

User:Lucifer &amp;#38; Viktor

User talk:Lucifer &amp;#38; Viktor

The Lord of the Rings

User:Cyberpower678/FaQs#InternetArchiveBot

User:Cyberbot II

User talk:Cyberbot II

The Lord of the Rings

User:Cyberpower678/FaQs#InternetArchiveBot

User:InternetArchiveBot

User talk:InternetArchiveBot

User:Peter K Burian

User talk:Peter K Burian

User:Deagol2

User talk:Deagol2

Wikipedia:Signatures

User talk:31.53.1.161#top

nation

Madsen Pirie

Biteback Publishing

Tax Foundation

government

Government debt

Inflation

Leap year

United States

W

Los Angeles, California

K. Eric Drexler

Engines of Creation

nanotechnology

Foresight Institute

cryonics

Alcor Life Extension Foundation

Max More

Extropy Institute

memeplex

cyberculture

David Pearce (philosopher)

World Transhumanist Association

scientific inquiry

Policy

Humanity+

futurist

human enhancement

transhumanist politics

Right-libertarianism

Modern liberalism in the United States

centre-left

James Hughes (sociologist)

Humanity+

Mormon Transhumanist Association

Giuseppe Vatinno

posthumanism

Conservatism

Christianity

Progressivism

biological uplift

artificial intelligence

Hans Moravec

apocalypticism

cultural posthumanism

the arts

Agency (philosophy)

prerogative

posthuman

humanism

Age of Enlightenment

Secular humanism

freethought

technocentrism

death

social justice

social change

Narcissism

Self-transcendence

Perfectionism (philosophy)

technoself studies

File:PPTCountdowntoSingularityLog.jpg

reason

biocentrism (ethics)

Congenital


pin tumbler lock

Doctor Who

Blink (Doctor Who)

ankh

Terrance Dicks

Malcolm Hulke

Doctor Who (1996 film)

Ghost Light (Doctor Who)

Survival (Doctor Who)

Rassilon

Pin tumbler lock

Blink (Doctor Who)

Weeping Angel

First Doctor

The Web Planet

The Daleks' Master Plan

Utopia (Doctor Who)

The Sensorites

Spearhead from Space

New Series Adventures (Doctor Who)

Only Human (Doctor Who)

meson

Genghis Khan

Rose (Doctor Who episode)

Doomsday (Doctor Who)

The War Games

Utopia (Doctor Who)

sonic screwdriver

Clara Oswald

Doctor Who (series 4)

River Song (Doctor Who)

The Eleventh Hour (Doctor Who)

The Caretaker (Doctor Who)

Clara Oswald

The Day of the Doctor

The End of Time (Doctor Who)

Master (Doctor Who)

Silver Nemesis

The Shakespeare Code

Gridlock (Doctor Who)

Utopia (Doctor Who)

Jack Harkness

The Time of the Doctor

Vincent and the Doctor

Face the Raven

Hell Bent (Doctor Who)

The Ribos Operation

Arc of Infinity

The Web Planet

Death to the Daleks

Mandr

In [38]:
precontext_dict = {}
for link in matches:
    precontext_dict[link.group(4)] = link.group(1)
    
with open(PATH_WIKI_OUT+ 'precontext_dict'+'.p', 'wb') as fp:
    pickle.dump(precontext_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)